In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [3]:
df=pd.read_csv("Breast_cancer_dataset.csv")
del df["Unnamed: 32"]
goal_col = df.pop("diagnosis")
df = df.set_index("id")
x_train, x_test, y_train, y_test = train_test_split(df, goal_col, test_size=0.2, random_state=42,stratify=goal_col)








In [4]:
class GaussianDiscriminantAnalysis:
    def __init__(self,train_data,train_label):
        self.x_train = train_data
        self.y_train = train_label
        positive_nums = 0
        negative_nums = 0
        positive_data = []
        negetive_data = []
        for (data,label) in zip(train_data.values,train_label.values):
            if label == "M":
                positive_nums += 1
                positive_data.append(data)
            else:
                negative_nums += 1
                negetive_data.append(data)
        positive_data = np.array(positive_data)
        negetive_data = np.array(negetive_data)
        self.positive_nums = positive_nums
        self.negative_nums = negative_nums
        self.positive_data = positive_data
        self.negative_data = negetive_data
        self.positive_mean = np.mean(self.positive_data,axis=0)
        self.negative_mean = np.mean(self.negative_data,axis=0)
        self.positive_covariance = np.cov(self.positive_data,rowvar=False)
        self.negative_covariance = np.cov(self.negative_data,rowvar=False)
        self.positive_possibility = positive_nums/(positive_nums+negative_nums)
        self.negative_possibility = negative_nums/(positive_nums+negative_nums)
        self.covariance = ((positive_nums-1)*self.positive_covariance+(negative_nums-1)*self.negative_covariance)/(positive_nums+negative_nums-2)
        
    def Gaussian(self,x,mean,covariance):
        d = x.shape[0]
        covariance_inv = np.linalg.inv(covariance)
        numerator = np.exp(-0.5 * (x - mean).T @ covariance_inv @ (x - mean))
        denominator = np.sqrt((2 * np.pi) ** d * np.linalg.det(covariance))
        return numerator / denominator

    def predict(self,test_data):
        predictions = []
        for data in test_data.values:
            positive_prob = self.Gaussian(data, self.positive_mean, self.positive_covariance) * self.positive_possibility
            negative_prob = self.Gaussian(data, self.negative_mean, self.negative_covariance) * self.negative_possibility
            if positive_prob > negative_prob:
                predictions.append("M")
            else:
                predictions.append("B")
        return predictions

    def score(self,test_data,test_label):
        predictions = self.predict(test_data)
        return accuracy_score(test_label, predictions)

    def predict_with_idx(self,test_data):
        predictions = self.predict(test_data)
        return pd.Series(predictions, index=test_data.index)



In [6]:
gda = GaussianDiscriminantAnalysis(x_train,y_train)
# print(gda.predict(x_test))
print("gda准确率为:",gda.score(x_test,y_test))
print(gda.predict_with_idx(x_test))
# print(gda.predict_with_idx(x_test)[862548])
lr = LogisticRegression(max_iter=10000)
lr.fit(x_train, y_train)
test_predictions = lr.predict(x_test)
print("lr准确率为:", accuracy_score(y_test, test_predictions))








gda准确率为: 0.9473684210526315
id
865137     B
884948     M
901303     M
862548     B
9112085    B
          ..
901836     B
919537     B
911150     B
879523     M
8610404    M
Length: 114, dtype: object
lr准确率为: 0.9385964912280702
